In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from builder import generate_summary, generate_structured_resume
from parser import parse_resume
import os
import json
import re
from langchain_groq import ChatGroq
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from typing import Dict, Any
from dotenv import load_dotenv

c:\Shark_Stack\Projects\Resume_Builder\builder.py:44: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=SUMMARY_PROMPT)


In [4]:
# Load environment
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=GROQ_API_KEY, model_name="qwen/qwen3-32b")
# Prompt templates
SUMMARY_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "act as an professional resume writer. Generate a concise 2-3 sentence summary from the user’s free-form description. ONLY GENERATE THE SUMMARY NO ADDITIONAL TEXT OR HEADING SO I CAN JUST COPY PASTE IT"),
    ("human", "{raw_text}")
])

In [5]:
summary_chain = LLMChain(llm=llm, prompt=SUMMARY_PROMPT)
# struct_chain = LLMChain(llm=llm, prompt=STRUCT_PROMPT)


def generatee_summary(raw_text: str) -> str:
    result = summary_chain.predict(raw_text=raw_text)
    return result.strip()

In [6]:
import re
# Sample free-form text
raw_text = """
PROFILE
I am a Computer Science graduate from DHA Suffa University with experience in Machine Learning, Artificial Intelligence, NLP, and
Retrieval-Augmented Generation (RAG) applications. I have hands-on expertise in developing advanced models, including a characterlevel transformer for text generation and an NLP-based sentiment analysis tool achieving 92% accuracy. I have completed the Machine
Learning Specialization by Stanford Online and DeepLearning.AI , strengthening my technical foundation.
In addition to AI and NLP, I have experience in mobile development using Flutter and Dart, creating visually appealing and
functional cross-platform applications. As a C++ developer, I contributed to a national-level project at SUPARCO, where I optimized
parallel computing algorithms to significantly reduce runtime. Proficient in Python, C++, Flutter, and Rust, I excel in analytical
problem-solving and delivering impactful, real-world solutions.
SKILLS AI/ML RAG NLP ML Algorithms Flutter Pytorch Streamlit Python C ++ Dart
EXPERIENCE
LusionTech, AI Business Development Manager
AI Sales Management, Projects, and Communication with International Clients
2025/01 – present
Karachi, Pakistan
•Engaged with international clients through strategic meetings and personalized consultations to
deliver AI Solutions aligned with their unique business needs.
•Managed and oversaw all client-assigned AI projects, ensuring timely execution and high-quality
outcomes.
SUPARCO, AI Engineer
Contract-based job
2025/04 – present
Karachi, Pakistan
•Developing an AI-based data fusion system that integrates optical and SAR imagery transmitted from
an airborne observatory to the ground.
Research Based Project
•Physically Based Phase Function in Volume Rendering
2023/03 – present
•Objective: Enhance volumetric rendering quality.
•Focus: Propose our phase function, which is more physically accurate, showcasing applicability in
various use cases, including medical CT scans, MRIs, and animations.
SUPARCO, C++ developer
•SAR(Synthetic aperture radar) Image formation
2024/01 – 2024/05
karachi, Pakistan
•Develop, convert, optimize algorithms and optimize the code to run parallel on the CPU
"""
summary = generatee_summary(raw_text)
cleaned_summary = re.sub(r'<think>.*?</think>', '', summary, flags=re.DOTALL).strip()
print("Generated Summary:\n", cleaned_summary)


Generated Summary:
 Computer Science graduate with expertise in AI, ML, and NLP, specializing in RAG applications and advanced model development (e.g., 92% accurate sentiment analysis, character-level transformers). Proven experience in AI project management, cross-platform mobile development (Flutter/Dart), and optimizing high-performance computing solutions (C++, parallel algorithms) for national projects at SUPARCO, alongside leading AI sales and client solutions at LusionTech. Recognized for delivering impactful, technically rigorous solutions across AI research, engineering, and business development.


In [7]:
# Load JSON schema content to include in prompts
schema_path = os.path.join('models/resume_schema.json')
with open(schema_path) as f:
    raw_schema = f.read()
# Escape braces to avoid template interpolation
escaped_schema = raw_schema.replace('{', '{{').replace('}', '}}')

In [8]:
STRUCT_PROMPT = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a professional resume writer. Produce a JSON matching the following JSON schema exactly (no extra keys):\n"
        + escaped_schema
    )),
    ("human", (
        "Name: {name}\n"
        "Email: {email}\n"
        "Phone: {phone}\n\n"
        "Summary: {summary}\n\n"
        "Free-form description:\n{raw_text}"
    ))
])

struct_chain = LLMChain(llm=llm, prompt=STRUCT_PROMPT)


In [9]:
def generate_structured_resumee(profile: Dict[str, Any], summary: str, raw_text: str) -> str:
    result = struct_chain.predict(
        name=profile['personal_info']['name'],
        email=profile['personal_info']['email'],
        phone=profile['personal_info'].get('phone', ''),
        summary=summary,
        raw_text=raw_text
    )
    return result.strip()

In [10]:
profile = {
    'personal_info': {'name': 'Salman', 'email': 'salman313@gmail.com', 'phone': '123-456-7890'}
}
raw_json = generate_structured_resumee(profile, cleaned_summary, raw_text)
cleaned_json = re.sub(r'<think>.*?</think>', '', raw_json, flags=re.DOTALL).strip()
print("Raw JSON Resume:\n", cleaned_json)


Raw JSON Resume:
 ```json
{
  "title": "Resume",
  "personal_info": {
    "name": "Salman",
    "email": "salman313@gmail.com",
    "phone": "123-456-7890"
  },
  "summary": "Computer Science graduate with expertise in AI, ML, and NLP, specializing in RAG applications and advanced model development (e.g., 92% accurate sentiment analysis, character-level transformers). Proven experience in AI project management, cross-platform mobile development (Flutter/Dart), and optimizing high-performance computing solutions (C++, parallel algorithms) for national projects at SUPARCO, alongside leading AI sales and client solutions at LusionTech. Recognized for delivering impactful, technically rigorous solutions across AI research, engineering, and business development.",
  "skills": [
    "AI/ML",
    "RAG",
    "NLP",
    "ML Algorithms",
    "Flutter",
    "PyTorch",
    "Streamlit",
    "Python",
    "C++",
    "Dart"
  ],
  "experience": [
    {
      "role": "AI Business Development Manager",

In [11]:
def clean_raw_json(s: str) -> str:
    # Remove markdown fences and any leading/trailing text
    s = s.strip()
    # Remove ```json header/footer if present
    if s.startswith("```"):
        s = "\n".join(s.splitlines()[1:-1])
    return s

raw_json = clean_raw_json(cleaned_json)

In [26]:
raw_json

'{\n  "personal_info": {\n    "name": "Alice Smith",\n    "email": "alice@example.com",\n    "phone": "123-456-7890"\n  },\n  "summary": "Computer Science graduate with expertise in AI/ML, NLP, and RAG, demonstrated through hands-on projects like a 92% accurate sentiment analysis tool and a character-level transformer model. Proficient in Python, C++, Flutter, and Rust, with experience leading AI initiatives at LusionTech, engineering data fusion systems at SUPARCO, and optimizing parallel computing algorithms. Blends technical depth with real-world application, supported by advanced training from Stanford and DeepLearning.AI.",\n  "skills": [\n    "AI/ML",\n    "RAG",\n    "NLP",\n    "ML Algorithms",\n    "Flutter",\n    "Pytorch",\n    "Streamlit",\n    "Python",\n    "C++",\n    "Dart"\n  ],\n  "experience": [\n    {\n      "role": "AI Business Development Manager",\n      "company": "LusionTech",\n      "start_date": "2025/01",\n      "end_date": "present",\n      "description": "

In [12]:
import json
from pydantic import BaseModel, EmailStr
from typing import List, Optional

class PersonalInfo(BaseModel):
    name: str
    email: EmailStr
    phone: Optional[str] = None

class ExperienceItem(BaseModel):
    role: str
    company: str
    start_date: str
    end_date: Optional[str] = None  # made optional with default
    description: str

class Resume(BaseModel):
    personal_info: PersonalInfo
    summary: str
    skills: List[str]
    experience: Optional[List[ExperienceItem]] = None  # optional list


def parse_resumee(raw_json_str: str) -> Resume:
    data = json.loads(raw_json_str)
    return Resume(**data)

In [13]:
resume = parse_resumee(raw_json)
# print("Parsed Resume Object:\n", resume.json(indent=2))

In [14]:
resume

Resume(personal_info=PersonalInfo(name='Salman', email='salman313@gmail.com', phone='123-456-7890'), summary='Computer Science graduate with expertise in AI, ML, and NLP, specializing in RAG applications and advanced model development (e.g., 92% accurate sentiment analysis, character-level transformers). Proven experience in AI project management, cross-platform mobile development (Flutter/Dart), and optimizing high-performance computing solutions (C++, parallel algorithms) for national projects at SUPARCO, alongside leading AI sales and client solutions at LusionTech. Recognized for delivering impactful, technically rigorous solutions across AI research, engineering, and business development.', skills=['AI/ML', 'RAG', 'NLP', 'ML Algorithms', 'Flutter', 'PyTorch', 'Streamlit', 'Python', 'C++', 'Dart'], experience=[ExperienceItem(role='AI Business Development Manager', company='LusionTech', start_date='2025-01', end_date='present', description='Engaged with international clients through

In [15]:
import argparse
import json
from builder import generate_summary, generate_structured_resume
from parser import parse_resume
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors

def ssave_outputss(resume, raw_json_path, md_path, pdf_path):
    # 1) JSON + MD as before...
    raw_json = resume.json()
    with open(raw_json_path, 'w') as f:
        f.write(raw_json)
    with open(md_path, 'w') as f:
        f.write(f"# {resume.personal_info.name}\n\n")
        f.write(f"**Email:** {resume.personal_info.email}  \n")
        if resume.personal_info.phone:
            f.write(f"**Phone:** {resume.personal_info.phone}  \n")
        f.write("---\n\n")
        f.write(f"## Summary\n{resume.summary}\n\n")
        f.write("## Skills\n" + "\n".join(f"- {s}" for s in resume.skills) + "\n\n")
        if resume.experience:
            f.write("## Experience\n")
            for e in resume.experience:
                f.write(f"**{e.role}**, {e.company} ({e.start_date} - {e.end_date or 'Present'})\n\n")
                f.write(f"{e.description}\n\n")

    # 2) Pretty PDF generation (adapted from generate_pdfs.py) :contentReference[oaicite:2]{index=2}
    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter
    margin = 50
    y = height - margin
    first_page = True

    def draw_line():
        nonlocal y
        c.line(margin, y, width - margin, y)
        y -= 20

    def add_wrapped_text(prefix=None, text="", font="Helvetica", size=10,
                         bold_prefix=False, indent=0):
        nonlocal y, first_page
        # Paginate if needed
        if y < margin + size + 10:
            c.showPage()
            y = height - margin
            first_page = False
            # no header on later pages

        c.setFont(font, size)
        full_text = f"{prefix}: {text}" if prefix else text
        words = full_text.split(" ")
        lines = []
        current = ""
        for w in words:
            test = (current + " " + w).strip()
            if c.stringWidth(test, font, size) <= (width - 2*margin - indent):
                current = test
            else:
                lines.append(current)
                current = w
        if current:
            lines.append(current)

        for line in lines:
            x = margin + indent
            if prefix and bold_prefix and line.startswith(f"{prefix}:"):
                # draw the prefix in bold, rest in normal
                c.setFont("Helvetica-Bold", size)
                prefix_text = f"{prefix}: "
                pw = c.stringWidth(prefix_text, "Helvetica-Bold", size)
                c.drawString(x, y, prefix_text)
                c.setFont(font, size)
                c.drawString(x + pw, y, line[len(prefix_text):])
            else:
                c.drawString(x, y, line)
            y -= size + 2

    def add_contact_info():
        nonlocal y
        c.setFont("Helvetica", 10)
        # phone | LinkedIn | Email
        contact = f"Phone: {resume.personal_info.phone or ''} | LinkedIn | Email"
        tw = c.stringWidth(contact, "Helvetica", 10)
        x = (width - tw) / 2

        # Phone
        c.drawString(x, y, f"Phone: {resume.personal_info.phone or ''}")
        x += c.stringWidth(f"Phone: {resume.personal_info.phone or ''} | ", "Helvetica", 10)

        # LinkedIn
        c.setFillColor(colors.blue)
        c.drawString(x, y, "LinkedIn")
        # assuming resume.personal_info.linkedin exists
        c.linkURL('https://www.linkedin.com/', 
                  (x, y-2, x + c.stringWidth("LinkedIn", "Helvetica", 10), y+10), 
                  relative=0)
        x += c.stringWidth("LinkedIn | ", "Helvetica", 10)

        # Email
        c.drawString(x, y, "Email")
        c.linkURL(f"mailto:{resume.personal_info.email}",
                  (x, y-2, x + c.stringWidth("Email", "Helvetica", 10), y+10),
                  relative=0)
        c.setFillColor(colors.black)
        y -= 15

    # First‐page header
    c.setFont("Helvetica-Bold", 16)
    header_x = (width - c.stringWidth(resume.personal_info.name, "Helvetica-Bold", 16)) / 2
    c.drawString(header_x, y, resume.personal_info.name)
    y -= 30

    # Contact line + divider
    add_contact_info()
    draw_line()

    # Summary
    add_wrapped_text("Summary", resume.summary, size=12, bold_prefix=True)
    draw_line()

    # (Skip Education section if you haven’t modeled it; otherwise:)
    # add_wrapped_text("Education", "", size=12, bold_prefix=True)
    # for edu in resume.education or []:
    #     add_wrapped_text(f"{edu.degree} - {edu.university}", 
    #                      f"GPA: {edu.gpa} ({edu.graduation_date})")
    # draw_line()

    # Technical Skills
    add_wrapped_text("Technical Skills", "", size=12, bold_prefix=True)
    add_wrapped_text("Skills", ", ".join(resume.skills), bold_prefix=True)
    draw_line()

    # Work Experience
    add_wrapped_text("Work Experience", "", size=12, bold_prefix=True)
    for exp in resume.experience or []:
        title = f"{exp.role} - {exp.company} ({exp.start_date} - {exp.end_date or 'Present'})"
        add_wrapped_text(title, "", bold_prefix=True)
        for bullet in exp.description.split(". "):
            if bullet.strip():
                add_wrapped_text("-", bullet.strip(), indent=10)
    draw_line()

    # Project Experience (if present)
    # add_wrapped_text("Project Experience", "", size=12, bold_prefix=True)
    # for proj in getattr(resume, "projects", []) or []:
    #     add_wrapped_text(proj.name, proj.description, bold_prefix=True)

    # Finish up
    c.save()
    print(f"Generated: {raw_json_path}, {md_path}, {pdf_path}")


def interactive_mode(md_path, raw_json_path, pdf_path):
    name = input('Name: ')
    email = input('Email: ')
    phone = input('Phone (optional): ')
    print('Enter skills & experience paragraphs. End with blank line:')
    lines=[]
    while True:
        l = input()
        if not l.strip(): break
        lines.append(l)
    raw_text='\n'.join(lines)

    summary = generate_summary(raw_text)
    profile={'personal_info':{'name':name,'email':email,'phone':phone},'skills':[], 'experience':[]}
    # skills & experience are parsed from the structured JSON
    raw_json_str = generate_structured_resume(profile, summary, raw_text)
    resume = parse_resume(raw_json_str)
    save_outputs(resume, raw_json_path, md_path, pdf_path)
    print(f"Outputs: {md_path}, {raw_json_path}, {pdf_path}")


def file_mode(input_path, md_path, raw_json_path, pdf_path):
    data = json.load(open(input_path))
    raw_text = data.pop('raw_text','')
    summary = data.get('summary') or generate_summary(raw_text)
    raw_json_str = generate_structured_resume(data, summary, raw_text)
    resume = parse_resume(raw_json_str)
    save_outputs(resume, raw_json_path, md_path, pdf_path)

# if __name__=='__main__':
#     parser=argparse.ArgumentParser()
#     group=parser.add_mutually_exclusive_group(required=True)
#     group.add_argument('--interactive',action='store_true')
#     group.add_argument('--input',type=str)
#     parser.add_argument('--output-md',default='resume.md')
#     parser.add_argument('--output-json',default='raw_resume.json')
#     parser.add_argument('--output-pdf',default='resume.pdf')
#     args=parser.parse_args()

#     if args.interactive:
#         interactive_mode(args.output_md, args.output_json, args.output_pdf)
#     else:
#         file_mode(args.input, args.output_md, args.output_json, args.output_pdf)

In [16]:
# from main import save_outputss
# Save to files|
ssave_outputss(resume, 'test_resume.json', 'test_resume.md', 'test_resume.pdf')
print("Artifacts generated: test_resume.json, test_resume.md, test_resume.pdf")

Generated: test_resume.json, test_resume.md, test_resume.pdf
Artifacts generated: test_resume.json, test_resume.md, test_resume.pdf


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_22656\716140464.py:14: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  raw_json = resume.json()
